In [2]:
pip install graphistry

     |████████████████████████████████| 127 kB 4.0 MB/s            
     |████████████████████████████████| 20.2 MB 6.4 MB/s            
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import graphistry

graphistry.register(api=1, protocol="https", server="hub.graphistry.com", key="4f55eede43c4020fbe604f4bb159737568e7e890c202fc943f2164dd1bdc4adf")


In [9]:
# load data
pd_data = pd.read_table('kg_without_inconsistencies.txt', sep='\t')
pd_data = pd_data[['Subject', 'Predicate', 'Object']]
pd_data.head()


,Subject,Predicate,Object
0,AAC(3)-Ia,confers resistance to antibiotic,Aminoglycosides
1,AAC(3)-Ia,confers resistance to antibiotic,Gentamicin
2,AAC(3)-Ia,confers resistance to antibiotic,Streptomycin
3,AAC(3)-Ia,has,response to antibiotic
4,AAC(3)-VIa,confers resistance to antibiotic,Aminoglycosides


In [12]:

# filter positives only
neg_predicates = [
    'confers no resistance to antibiotic',
    'is not regulated by antibiotic',
    'is not mutated by antibiotic',
    'no represses',
    'no activates',]

def _check_match(x, predicates):
    flag = False
    for predicate in predicates:
        if predicate in x:
            flag = True
    return flag

pd_pos_data = pd_data[~pd_data['Predicate'].apply(lambda x: _check_match(x, neg_predicates))]

# assign predicate category
def map_func(label):
    if 'confers resistance to antibiotic' in label:
        return 'confers resistance to antibiotic'
    elif 'upregulated by antibiotic' in label:
        return 'upregulated by antibiotic'
    elif 'represses' in label:
        return 'represses'
    elif 'activates' in label:
        return 'activates'
    elif 'has' in label:
        return 'has'
    elif 'is involved in' in label:
        return 'is involved in'
    elif 'is part of' in label:
        return 'is part of'
    elif 'targeted by' in label:
        return 'targeted by'
    elif 'is mutated by antibiotic' in label:
        return 'is mutated by antibiotic'
    elif 'is associated with resistance to antibiotic' in label:
        return 'is associated with resistance to antibiotic'
    elif 'is downregulated by antibiotic' in label:
        return 'is downregulated by antibiotic'
    
    else:
        raise ValueError('Invalid label: {}'.format(label))

pd_pos_data['Predicate Category'] = pd_pos_data['Predicate'].apply(lambda x: map_func(x))

###/home/jyoun/.local/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
###A value is trying to be set on a copy of a slice from a DataFrame.
###Try using .loc[row_indexer,col_indexer] = value instead


#See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-12-ec18568fb1bf>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_pos_data['Predicate Category'] = pd_pos_data['Predicate'].apply(lambda x: map_func(x))


In [15]:

g = graphistry.bind(
    source='Subject',
    destination='Object',
    edge_label='Predicate',)

g.plot(pd_pos_data)

In [16]:
VIEW = "10"
EDIT = "20"
graphistry.privacy(
  mode='private',
  invited_users=[
    {"email": "asalim@ucdavis.edu", "action": VIEW},
    {"email": "asalim@ucdavis.edu", "action": EDIT}
  ],
  notify=True)